# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [6]:
# import libraries
import re
import pickle
import pandas as pd 
from sqlalchemy import create_engine
import nltk 
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split,  GridSearchCV 
from sklearn.metrics import classification_report


nltk.download(['wordnet', 'punkt', 'stopwords'])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Huawei\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Huawei\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Huawei\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
# load data from database

engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName',engine)

X = df['message']  # Message Column
Y = df.iloc[:, 4:] # Classification label

### 2. Write a tokenization function to process your text data

In [16]:


def tokenize(text):
    """
    Function: split text into words and return the root form of the words
    Args:
      text(str): the message
    Return:
      lemm(list of str): a list of the root form of the message words
    """
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize text
    words = word_tokenize(text)
    
    # Remove stop words
    stop = stopwords.words("english")
    words = [t for t in words if t not in stop]
    
    # Lemmatization
    lemm = [WordNetLemmatizer().lemmatize(w) for w in words]
    return lemm

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [17]:
# Pipleine 1: Random Forest Classifier

pipeline_rfc = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(RandomForestClassifier()))
    ])


# Pipeline 2: Naive Bayes classifier

pipeline_nbc = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(MultinomialNB()))
    ])


# Pipleine 3: Adaboost Classifier 

pipeline_ada = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [18]:

# Spiliting data
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [19]:
# Fit the Random Forest Classifier 
pipeline_rfc.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000236662694C0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [20]:
# Fit the Naive Bayes classifier 
pipeline_nbc.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000236662694C0>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=MultinomialNB()))])

In [21]:
# Fit the Adaboost Classifier 
pipeline_ada.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000236662694C0>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [22]:
def plot_scores(y_test, y_pred):
    #Testing the model
    # Printing the classification report for each label
    i = 0
    for col in y_test:
        print('Feature {}: {}'.format(i+1, col))
        print(classification_report(y_test[col], y_pred[:, i]))
        i = i + 1
    accuracy = (y_pred == y_test.values).mean()
    print('The model accuracy is {:.3f}'.format(accuracy))

In [23]:
# Prediction: the Random Forest Classifier  
y_pred = pipeline_rfc.predict(X_test)
plot_scores(y_test, y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.72      0.43      0.54      1563
           1       0.84      0.94      0.89      4945
           2       0.33      0.46      0.39        46

    accuracy                           0.82      6554
   macro avg       0.63      0.61      0.60      6554
weighted avg       0.81      0.82      0.80      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5400
           1       0.83      0.52      0.64      1154

    accuracy                           0.90      6554
   macro avg       0.87      0.75      0.79      6554
weighted avg       0.89      0.90      0.89      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.00      0.00      0.00        31

    accuracy                           1.00      6554
   macro avg       

C:\Users\Huawei\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6436
           1       0.87      0.11      0.20       118

    accuracy                           0.98      6554
   macro avg       0.93      0.55      0.59      6554
weighted avg       0.98      0.98      0.98      6554

Feature 15: money
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6410
           1       1.00      0.03      0.05       144

    accuracy                           0.98      6554
   macro avg       0.99      0.51      0.52      6554
weighted avg       0.98      0.98      0.97      6554

Feature 16: missing_people
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6482
           1       1.00      0.01      0.03        72

    accuracy                           0.99      6554
   macro avg       0.99      0.51      0.51      6554
weighted avg       0.99      

In [24]:
# Prediction: the Naive Bayes classifier 
y_pred = pipeline_nbc.predict(X_test)
plot_scores(y_test, y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.80      0.09      0.16      1563
           1       0.77      0.99      0.87      4945
           2       0.00      0.00      0.00        46

    accuracy                           0.77      6554
   macro avg       0.52      0.36      0.34      6554
weighted avg       0.77      0.77      0.69      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.86      0.99      0.92      5400
           1       0.89      0.22      0.35      1154

    accuracy                           0.86      6554
   macro avg       0.87      0.61      0.63      6554
weighted avg       0.86      0.86      0.82      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.00      0.00      0.00        31

    accuracy                           1.00      6554
   macro avg       

C:\Users\Huawei\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6436
           1       0.00      0.00      0.00       118

    accuracy                           0.98      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.96      0.98      0.97      6554

Feature 24: tools
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6516
           1       0.00      0.00      0.00        38

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

Feature 25: hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6508
           1       0.00      0.00      0.00        46

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99 

In [25]:
# Prediction: the Adaboost Classifier 
y_pred = pipeline_ada.predict(X_test)
plot_scores(y_test, y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.70      0.14      0.23      1563
           1       0.78      0.98      0.87      4945
           2       0.33      0.13      0.19        46

    accuracy                           0.77      6554
   macro avg       0.60      0.42      0.43      6554
weighted avg       0.76      0.77      0.71      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.90      0.97      0.93      5400
           1       0.77      0.53      0.62      1154

    accuracy                           0.89      6554
   macro avg       0.84      0.75      0.78      6554
weighted avg       0.88      0.89      0.88      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.13      0.06      0.09        31

    accuracy                           0.99      6554
   macro avg       

### 6. Improve your model
Use grid search to find better parameters. 

In [26]:
# Show parameters for the pipline
pipeline_rfc.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x00000236662694C0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x00000236662694C0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [27]:
# Show parameters for the pipline
pipeline_ada.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x00000236662694C0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x00000236662694C0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=AdaBoostClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf_

In [28]:
# Using grid search
# Create Grid search parameters for Random Forest Classifier   
parameters_rfc = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [10, 20]
}

cv_rfc = GridSearchCV(pipeline_rfc, param_grid = parameters_rfc)
cv_rfc

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000236662694C0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [10, 20],
                         'tfidf__use_idf': (True, False)})

In [29]:
# Using grid search
# Create Grid search parameters
parameters_ada = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 60, 70]
}

cv_ada = GridSearchCV(pipeline_ada, param_grid = parameters_ada)
cv_ada

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000236662694C0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [50, 60, 70],
                         'tfidf__use_idf': (True, False)})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [30]:
# Fit the first tuned model
cv_rfc.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000236662694C0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [10, 20],
                         'tfidf__use_idf': (True, False)})

In [31]:
# Fit the second tuned model
cv_ada.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000236662694C0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [50, 60, 70],
                         'tfidf__use_idf': (True, False)})

In [32]:
# Predicting using the first tuned model 
y_pred = cv_rfc.predict(X_test)
plot_scores(y_test, y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.70      0.43      0.53      1563
           1       0.84      0.94      0.88      4945
           2       0.37      0.41      0.39        46

    accuracy                           0.81      6554
   macro avg       0.63      0.59      0.60      6554
weighted avg       0.80      0.81      0.80      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.90      0.97      0.94      5400
           1       0.81      0.49      0.61      1154

    accuracy                           0.89      6554
   macro avg       0.85      0.73      0.77      6554
weighted avg       0.88      0.89      0.88      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.00      0.00      0.00        31

    accuracy                           1.00      6554
   macro avg       

C:\Users\Huawei\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6261
           1       0.68      0.09      0.15       293

    accuracy                           0.96      6554
   macro avg       0.82      0.54      0.56      6554
weighted avg       0.95      0.96      0.94      6554

Feature 22: buildings
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6276
           1       0.72      0.17      0.27       278

    accuracy                           0.96      6554
   macro avg       0.84      0.58      0.62      6554
weighted avg       0.95      0.96      0.95      6554

Feature 23: electricity
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6436
           1       0.77      0.08      0.15       118

    accuracy                           0.98      6554
   macro avg       0.88      0.54      0.57      6554
weighted avg       0.98     

In [33]:

# Predicting using the second tuned model 
y_pred = cv_ada.predict(X_test)
plot_scores(y_test, y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.72      0.08      0.15      1563
           1       0.77      0.99      0.86      4945
           2       0.32      0.20      0.24        46

    accuracy                           0.77      6554
   macro avg       0.61      0.42      0.42      6554
weighted avg       0.76      0.77      0.69      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.91      0.96      0.93      5400
           1       0.76      0.53      0.63      1154

    accuracy                           0.89      6554
   macro avg       0.83      0.75      0.78      6554
weighted avg       0.88      0.89      0.88      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.20      0.06      0.10        31

    accuracy                           0.99      6554
   macro avg       

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [34]:
# Fit the second tuned model
cv_ada.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000236662694C0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [50, 60, 70],
                         'tfidf__use_idf': (True, False)})

### 9. Export your model as a pickle file

In [36]:
# Create a pickle file for the model
file_name = 'classifier.pkl'
with open (file_name, 'wb') as f:
    pickle.dump(cv_ada, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.